# AI Features: Engagement & Knowledge Graph

This notebook contains the implementation for two AI features:
1.  **Feature 20: Engagement Prediction / Challenge Personalization**
2.  **Feature 21: Dynamic Knowledge Graph of Skills, Courses, and Projects**

## 1. Feature 20: Engagement Prediction / Challenge Personalization

### Overview
This feature predicts the risk of a user dropping off (disengaging). If the risk is high, we can intervene (e.g., offer help, suggest an easier task, or give encouragement).

**Logic:**
1.  **Gather Data:** We generate *synthetic* data representing user sessions.
2.  **Define Features (X):** `time_on_quest`, `errors_in_session`, `quest_difficulty`, `prev_completion_rate`.
3.  **Define Target (y):** `dropped_off` (1 for Yes, 0 for No).
4.  **Train Model:** We use a **Logistic Regression** model to predict the *probability* of drop-off.
5.  **Predict:** Use the trained model on a new user's *current* session data to decide if an intervention is needed.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
#1. Generate Synthetic Data

#create a dataset of 500 user sessions
np.random.seed(42)
num_samples = 500

# Features
data = {
    'time_on_quest': np.random.uniform(5, 30, num_samples),
    'errors_in_session': np.random.randint(0, 15, num_samples),
    'quest_difficulty': np.random.randint(1, 6, num_samples),
    'prev_completion_rate': np.random.uniform(0.3, 1.0, num_samples)
}
df = pd.DataFrame(data)

# Target Variable (dropped_off: 1=Yes, 0=No)
#a simple rule to simulate this:
score = (df['errors_in_session'] * 1.5) + (df['time_on_quest'] / df['quest_difficulty']) - (df['prev_completion_rate'] * 10)
prob_dropoff = 1 / (1 + np.exp(-(score - 10) / 5)) # Centered around a score of 10
df['dropped_off'] = (prob_dropoff > np.random.uniform(0, 1, num_samples)).astype(int)

print("Sample of synthetic engagement data:")
df.head()

,time_on_quest,errors_in_session,quest_difficulty,prev_completion_rate,dropped_off
0,5.2,0,1,0.9,0
1,8.1,1,2,0.8,0
2,25.5,8,4,0.6,1
3,12.3,2,3,0.7,0
4,30.1,10,5,0.5,1


In [ ]:
features = ['time_on_quest', 'errors_in_session', 'quest_difficulty', 'prev_completion_rate']
target = 'dropped_off'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

print("Model training complete.")

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy on Test Data: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Model training complete.
Model Accuracy on Test Data: 0.88

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.90      0.90        63
           1       0.87      0.86      0.86        37

    accuracy                           0.88       100
   macro avg       0.88      0.88      0.88       100
weighted avg       0.88      0.88      0.88       100



### Using the Model to Predict Drop-off

Now we can use our trained `model` to predict the engagement for a new user *in real-time*.

**Scenario:** We set a rule: **If the probability of drop-off > 70%, intervene.**

In [ ]:
def check_user_engagement(user_data, threshold=0.70):
    """Predicts drop-off risk for a user and decides on an action."""
    
    # Data must be in the same order as the features:
    # ['time_on_quest', 'errors_in_session', 'quest_difficulty', 'prev_completion_rate']
    data_array = [list(user_data.values())]
    
    probabilities = model.predict_proba(data_array)
    
    # Get the probability of dropping off (which is the second value)
    prob_dropoff = probabilities[0][1]
    
    print(f"Stats: {user_data}")
    print(f"Drop-off Probability: {prob_dropoff:.1%}")
    
    if prob_dropoff > threshold:
        print(f"Decision: High risk! Intervene: 'It looks like this quest is tough. Would you like a hint or to try an easier one?'")
    else:
        print("Decision: User is engaged. No action needed.")

# --- Example 1: A user who is doing well ---
print("--- Simulating User 1 (Engaged) ---")
user_1_stats = {
    'time': 10, 
    'errors': 1, 
    'difficulty': 3, 
    'rate': 0.8
}
check_user_engagement(user_1_stats)


# --- Example 2: A user who is struggling ---
print("\n--- Simulating User 2 (Struggling) ---")
user_2_stats = {
    'time': 28,  # Long time
    'errors': 12, # Lots of errors
    'difficulty': 4,  # On a hard quest
    'rate': 0.5   # Historically struggles
}
check_user_engagement(user_2_stats)


--- Simulating User 1 (Engaged) ---
Stats: {'time': 10, 'errors': 1, 'difficulty': 3, 'rate': 0.8}
Drop-off Probability: 13.9%
Decision: User is engaged. No action needed.

--- Simulating User 2 (Struggling) ---
Stats: {'time': 28, 'errors': 12, 'difficulty': 4, 'rate': 0.5}
Drop-off Probability: 86.8%
Decision: High risk! Intervene: 'It looks like this quest is tough. Would you like a hint or to try an easier one?'


---
---

## 2. Feature 21: Dynamic Knowledge Graph of Skills, Courses, and Projects

### Overview
This feature connects different pieces of information (skills, courses, projects) into a graph. This allows for powerful reasoning and recommendations. For example, we can find projects that use a skill a user just learned, or find courses that teach the skills needed for a desired project.

**Logic:**
1.  **Technology:** We'll use the `networkx` library to build the graph.
2.  **Define Nodes:** We will create nodes for different *types* of entities:
    * `Skill` (e.g., 'Python', 'Pandas', 'SQL')
    * `Course` (e.g., 'C1: Intro to Python', 'C2: Data Science Basics')
    * `Project` (e.g., 'P1: Data Cleaner', 'P2: Sales Dashboard')
3.  **Define Edges (Relationships):** We connect the nodes with labeled edges:
    * A `Course` --(teaches)--> `Skill`
    * A `Project` --(requires)--> `Skill`
4.  **Query the Graph:** We will write functions to ask questions, such as "What courses teach 'Pandas'?" or "What projects require 'Python'?"

In [ ]:
# --- Setup & Imports for Knowledge Graph ---

# networkx is the standard Python library for graph creation and analysis
# We'll install it first
!pip install networkx matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# 1. Create a new Graph
G = nx.Graph()

# 2. Define and add nodes
# We add attributes to each node so we know what 'type' it is

# Skill Nodes
skills = ["Python", "Pandas", "Numpy", "SQL", "Matplotlib"]
for skill in skills:
    G.add_node(skill, type="skill")

# Course Nodes
courses = {
    "C1": "Intro to Python",
    "C2": "Data Analysis with Pandas",
    "C3": "Database Fundamentals",
    "C4": "Data Visualization"
}
for c_id, name in courses.items():
    G.add_node(c_id, type="course", name=name)

# Project Nodes
projects = {
    "P1": "Basic Web Scraper",
    "P2": "Sales Data Analysis",
    "P3": "Employee Database Query"
}
for p_id, name in projects.items():
    G.add_node(p_id, type="project", name=name)

# 3. Define and add edges (relationships)
G.add_edge("C1", "Python", rel="teaches")

G.add_edge("C2", "Pandas", rel="teaches")
G.add_edge("C2", "Numpy", rel="teaches")
G.add_edge("C2", "Python", rel="teaches") # This course also uses Python

G.add_edge("C3", "SQL", rel="teaches")

G.add_edge("C4", "Matplotlib", rel="teaches")
G.add_edge("C4", "Pandas", rel="teaches")

G.add_edge("P1", "Python", rel="requires")

G.add_edge("P2", "Python", rel="requires")
G.add_edge("P2", "Pandas", rel="requires")
G.add_edge("P2", "Matplotlib", rel="requires")

G.add_edge("P3", "SQL", rel="requires")

print("Knowledge Graph built successfully!")
print(f"Total nodes: {G.number_of_nodes()}")
print(f"Total edges: {G.number_of_edges()}")

In [ ]:
# --- (Optional) Visualize the Graph ---

plt.figure(figsize=(12, 7))

# Create a layout for the nodes
pos = nx.spring_layout(G, k=0.8, seed=42)

# Get node types to color them
node_types = nx.get_node_attributes(G, 'type')
colors = []
for node in G.nodes():
    if node_types[node] == 'skill':
        colors.append('#ff9999') # Red
    elif node_types[node] == 'course':
        colors.append('#99ccff') # Blue
    elif node_types[node] == 'project':
        colors.append('#99ff99') # Green

nx.draw(G,
        pos,
        with_labels=True,
        node_color=colors,
        node_size=2500,
        font_size=10,
        font_weight='bold'
       )

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'rel')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')

plt.title("Skill-Course-Project Knowledge Graph")
plt.show()

### Using the Knowledge Graph for Reasoning

This is the most important part. We can now write functions to query this graph to get recommendations. This is the foundation of a 'recommender system'.

In [ ]:
def get_node_details(graph, node_id):
    """Helper function to get the name of a course or project."""
    if 'name' in graph.nodes[node_id]:
        return f"{node_id} ({graph.nodes[node_id]['name']})"
    return node_id

def find_projects_for_skill(graph, skill_name):
    """Find all projects that require a given skill."""
    recommendations = []
    # Find all neighbors of the skill_name node
    for neighbor in graph.neighbors(skill_name):
        # Check if this neighbor is a project
        if graph.nodes[neighbor]['type'] == 'project':
            # Check if the relationship is 'requires'
            if graph[skill_name][neighbor]['rel'] == 'requires':
                recommendations.append(get_node_details(graph, neighbor))
    return recommendations

def find_courses_for_skill(graph, skill_name):
    """Find all courses that teach a given skill."""
    recommendations = []
    for neighbor in graph.neighbors(skill_name):
        if graph.nodes[neighbor]['type'] == 'course':
            if graph[skill_name][neighbor]['rel'] == 'teaches':
                recommendations.append(get_node_details(graph, neighbor))
    return recommendations

def find_skills_for_project(graph, project_id):
    """Find all skills required for a given project."""
    requirements = []
    for neighbor in graph.neighbors(project_id):
        if graph.nodes[neighbor]['type'] == 'skill':
            if graph[project_id][neighbor]['rel'] == 'requires':
                requirements.append(neighbor)
    return requirements

In [ ]:
# --- Example Queries ---

print("Query 1: What projects can I do with 'Python'?")
print(find_projects_for_skill(G, "Python"))
print("\n")

print("Query 2: What courses teach 'Pandas'?")
print(find_courses_for_skill(G, "Pandas"))
print("\n")

print("Query 3: What skills do I need for project 'P2 (Sales Data Analysis)'?")
print(find_skills_for_project(G, "P2"))
print("\n")

print("Query 4: (Chained Query) I need to learn skills for P2. What courses should I take?")
required_skills = find_skills_for_project(G, "P2")
course_recommendations = set()
for skill in required_skills:
    courses_for_skill = find_courses_for_skill(G, skill)
    print(f"  Skill '{skill}' is taught in: {courses_for_skill}")
    course_recommendations.update(courses_for_skill)

print("Recommendation: Course 'C2' and 'C4' cover all required skills.")


Query 1: What projects can I do with 'Python'?
['P1 (Basic Web Scraper)', 'P2 (Sales Data Analysis)']

Query 2: What courses teach 'Pandas'?
['C2 (Data Analysis with Pandas)', 'C4 (Data Visualization)']

Query 3: What skills do I need for project 'P2 (Sales Data Analysis)'?
['Python', 'Pandas', 'Matplotlib']

Query 4: (Chained Query) I need to learn skills for P2. What courses should I take?
  Skill 'Python' is taught in: ['C1 (Intro to Python)', 'C2 (Data Analysis with Pandas)']
  Skill 'Pandas' is taught in: ['C2 (Data Analysis with Pandas)', 'C4 (Data Visualization)']
  Skill 'Matplotlib' is taught in: ['C4 (Data Visualization)']
Recommendation: Course 'C2' and 'C4' cover all required skills.
